#### LeetCode - SQL - #1075

Refer [1075. Project Employees I](https://leetcode.com/problems/project-employees-i/description/)

Write an SQL query that reports the average experience years of all the employees for each project, rounded to 2 digits.

Return the result table in any order.

In [2]:
data_project = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
columns_project = ['project_id', 'employee_id']
schema_project = {'project_id':'Int64', 'employee_id':'Int64'}

data_employee = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
columns_employee = ['employee_id', 'name', 'experience_years']
schema_employee = {'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'}

In [ ]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName('LeetCode SQL').getOrCreate()

In [ ]:
# Create Spark Dataframe using Pandas
import pandas

pandas_df_project = pandas.DataFrame(data=data_project, columns=columns_project).astype(schema_project)
pandas_df_employee = pandas.DataFrame(data=data_employee, columns=columns_employee).astype(schema_employee)

df_project = spark_context.createDataFrame(pandas_df_project)
df_employee = spark_context.createDataFrame(pandas_df_employee)

df_project.printSchema()
df_employee.printSchema()

In [ ]:
# Create Spark Dataframe using PySpark functions - 1
schema_project = 'project_id integer, employee_id integer'
schema_employee = 'employee_id integer, name string, experience_years integer'

df_project = spark_context.createDataFrame(data=data_project, schema=schema_project)
df_employee = spark_context.createDataFrame(data=data_employee, schema=schema_employee)

df_project.printSchema()
df_employee.printSchema()

In [ ]:
# Create Spark Dataframe using PySpark functions - 2
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema_project = StructType([
    StructField('project_id', IntegerType(), False),
    StructField('employee_id', IntegerType(), False)
])
df_project = spark_context.createDataFrame(data=data_project, schema=schema_project)
df_project.printSchema()

schema_employee = StructType([
    StructField('employee_id', IntegerType(), False),
    StructField('name', StringType(), True),
    StructField('experience_years', IntegerType(), True)
])
df_employee = spark_context.createDataFrame(data=data_employee, schema=schema_employee)
df_employee.printSchema()

In [12]:
from pyspark.sql import functions as F

df_project_employee \
    = df_project.join(
        df_employee,
        df_project.employee_id == df_employee.employee_id,
        'inner'
    ).select(
        df_project.project_id,
        df_employee.employee_id,
        df_employee.experience_years
    )

df_project_employee.groupBy(df_project_employee.project_id).agg(
    F.round(F.avg(df_project_employee.experience_years), 2).alias('average_years')
).show()

+----------+-------------+
|project_id|average_years|
+----------+-------------+
|         1|          2.0|
|         2|          2.5|
+----------+-------------+
